In [7]:
import sys
# ^^^ pyforest auto-imports - don't write above this line
import numpy as np
import pandas as pd
import scanpy as sc

from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, normalized_mutual_info_score
from sklearn.cluster import KMeans, SpectralClustering
from sklearn.cluster import SpectralClustering

from sklearn.decomposition import PCA, SparsePCA, KernelPCA
from sklearn.manifold import TSNE

from rpy2.robjects import r, pandas2ri
from rpy2.robjects.vectors import StrVector

pandas2ri.activate()

import magic
import scprep

%matplotlib inline

# from sklearnex import patch_sklearn
# patch_sklearn()

import warnings

from sklearn.cluster import KMeans
from tqdm import tqdm

sys.path.insert(0, '../../imputation2/notebooks/repos/GNNImpute/')
from GNNImpute.api import GNNImpute

In [2]:
def get_cluster_metrics(pred, labels):
    ari_res = []
    ami_res = []
    nmi_res = []

#     try:
#         r.assign("data", pred.T)
#         seurat = r('''
#             countsData = data.frame(data)
#             pbmc <- CreateSeuratObject(counts = countsData, project = "thal_single_cell", min.cells = 1, min.features = 1)
#             pbmc <- FindVariableFeatures(pbmc, selection.method = "vst", verbose=FALSE)
#             all.genes <- rownames(pbmc)
#             pbmc <- ScaleData(pbmc, features = all.genes, verbose=FALSE)
#             pbmc <- RunPCA(pbmc, verbose=FALSE)
#             pbmc <- FindNeighbors(pbmc, verbose=FALSE)
#             pbmc <- FindClusters(pbmc, verbose=FALSE)
#             Idents(pbmc)
#         ''')
#         ari_res.append(adjusted_rand_score(labels, seurat))
#         ami_res.append(adjusted_mutual_info_score(labels, seurat))
#         nmi_res.append(normalized_mutual_info_score(labels, seurat))
#     except Exception as e:
#         pass

    pred_ = KMeans(n_clusters=len(np.unique(labels)), random_state=42).fit_predict(pred)

    ari_res.append(adjusted_rand_score(labels, pred_))
    ami_res.append(adjusted_mutual_info_score(labels, pred_))
    nmi_res.append(normalized_mutual_info_score(labels, pred_))

    warnings.filterwarnings("error")

    affinities = ['cosine', 'linear', 'poly']

    for i in affinities:
        try:
            pred_ = SpectralClustering(
                n_clusters=len(np.unique(labels)), 
                random_state=42, 
                affinity=i
            ).fit_predict(pred)
            ari_res.append(adjusted_rand_score(labels, pred_))
            ami_res.append(adjusted_mutual_info_score(labels, pred_))
            nmi_res.append(normalized_mutual_info_score(labels, pred_))
        except:
            ari_res.append(0)
            ami_res.append(0)
            nmi_res.append(0)

    warnings.resetwarnings()
    
    return max(ari_res), max(ami_res), max(nmi_res)

In [3]:
def get_data(i):
    df = pd.read_csv('../data/{}/data.csv.gz'.format(i), index_col=0)
    tmp = np.sign(df)
    cols = (np.sum(tmp) > int((df.shape[0])*0.05))
    rows = (np.sum(tmp, axis=1) > int((df.shape[1])*0.05))
    df = np.log(df.loc[rows, cols] + 1)
    df_norm = df.copy()
    df_norm = scprep.normalize.library_size_normalize(df_norm)    
    df_norm = scprep.transform.sqrt(df_norm)
    X_norm = pd.DataFrame(df_norm, columns=df.columns)
    labels = df.index
    return X_norm, labels

In [4]:
dir_list = !ls ../data/ | grep -v zip
dir_list

['baron',
 'bmcite',
 'brosens',
 'carey',
 'cbmc',
 'chang',
 'Fujii',
 'hcabm40k',
 'hrvatin',
 'jakel',
 'jiang',
 'loureiro',
 'manno',
 'mingyao',
 'pbmc3k',
 'Selewa',
 'Xu']

In [5]:
len(dir_list)

17

In [10]:
res = []
for i in tqdm(dir_list):
    X_norm, labels = get_data(i)

    adata = sc.AnnData(X_norm.values)
    adata = GNNImpute(
        adata=adata, layer='GATConv',
        no_cuda=False,
        d = '/export/scratch/inoue019/gnn/',
        epochs=3000, 
        lr=0.001, weight_decay=0.0005,
        hidden=50, patience=200,
        fastmode=True, heads=3,
        use_raw=False,
        verbose=True
    )
    
    pred = adata.X
    pred = pd.DataFrame(pred, columns=X_norm.columns, index=X_norm.index)

    res.append(get_cluster_metrics(pred, labels))

  0%|          | 0/17 [00:00<?, ?it/s]

Epoch: 0010 loss_train: 1.0006 loss_val: 0.9993
Epoch: 0020 loss_train: 0.9998 loss_val: 0.9985
Epoch: 0030 loss_train: 0.9981 loss_val: 0.9970
Epoch: 0040 loss_train: 0.9917 loss_val: 0.9909
Epoch: 0050 loss_train: 0.9834 loss_val: 0.9846
Epoch: 0060 loss_train: 0.9782 loss_val: 0.9811
Epoch: 0070 loss_train: 0.9730 loss_val: 0.9785
Epoch: 0080 loss_train: 0.9687 loss_val: 0.9730
Epoch: 0090 loss_train: 0.9667 loss_val: 0.9717
Epoch: 0100 loss_train: 0.9636 loss_val: 0.9708
Epoch: 0110 loss_train: 0.9637 loss_val: 0.9695
Epoch: 0120 loss_train: 0.9616 loss_val: 0.9689
Epoch: 0130 loss_train: 0.9609 loss_val: 0.9686
Epoch: 0140 loss_train: 0.9597 loss_val: 0.9676
Epoch: 0150 loss_train: 0.9594 loss_val: 0.9674
Epoch: 0160 loss_train: 0.9597 loss_val: 0.9678
Epoch: 0170 loss_train: 0.9598 loss_val: 0.9671
Epoch: 0180 loss_train: 0.9587 loss_val: 0.9680
Epoch: 0190 loss_train: 0.9588 loss_val: 0.9679
Epoch: 0200 loss_train: 0.9589 loss_val: 0.9681
Epoch: 0210 loss_train: 0.9588 loss_val:

  6%|▌         | 1/17 [00:32<08:38, 32.38s/it]

Epoch: 0010 loss_train: 0.9988 loss_val: 0.9972
Epoch: 0020 loss_train: 0.9937 loss_val: 0.9926
Epoch: 0030 loss_train: 0.9857 loss_val: 0.9849
Epoch: 0040 loss_train: 0.9754 loss_val: 0.9754
Epoch: 0050 loss_train: 0.9650 loss_val: 0.9652
Epoch: 0060 loss_train: 0.9554 loss_val: 0.9572
Epoch: 0070 loss_train: 0.9493 loss_val: 0.9509
Epoch: 0080 loss_train: 0.9451 loss_val: 0.9484
Epoch: 0090 loss_train: 0.9428 loss_val: 0.9460
Epoch: 0100 loss_train: 0.9418 loss_val: 0.9453
Epoch: 0110 loss_train: 0.9404 loss_val: 0.9439
Epoch: 0120 loss_train: 0.9399 loss_val: 0.9432
Epoch: 0130 loss_train: 0.9395 loss_val: 0.9435
Epoch: 0140 loss_train: 0.9391 loss_val: 0.9425
Epoch: 0150 loss_train: 0.9387 loss_val: 0.9423
Epoch: 0160 loss_train: 0.9383 loss_val: 0.9419
Epoch: 0170 loss_train: 0.9377 loss_val: 0.9420
Epoch: 0180 loss_train: 0.9378 loss_val: 0.9424
Epoch: 0190 loss_train: 0.9381 loss_val: 0.9425
Epoch: 0200 loss_train: 0.9376 loss_val: 0.9419
Epoch: 0210 loss_train: 0.9377 loss_val:

 12%|█▏        | 2/17 [02:02<16:31, 66.12s/it]

...

 88%|████████▊ | 15/17 [15:31<01:40, 50.24s/it]

Epoch: 0010 loss_train: 0.9968 loss_val: 1.0028
Epoch: 0020 loss_train: 0.9965 loss_val: 1.0026
Epoch: 0030 loss_train: 0.9960 loss_val: 1.0029
Epoch: 0040 loss_train: 0.9940 loss_val: 1.0029
Epoch: 0050 loss_train: 0.9883 loss_val: 0.9999
Epoch: 0060 loss_train: 0.9798 loss_val: 0.9978
Epoch: 0070 loss_train: 0.9737 loss_val: 0.9921
Epoch: 0080 loss_train: 0.9692 loss_val: 0.9912
Epoch: 0090 loss_train: 0.9647 loss_val: 0.9896
Epoch: 0100 loss_train: 0.9614 loss_val: 0.9914
Epoch: 0110 loss_train: 0.9577 loss_val: 0.9900
Epoch: 0120 loss_train: 0.9553 loss_val: 0.9915
Epoch: 0130 loss_train: 0.9526 loss_val: 0.9916
Epoch: 0140 loss_train: 0.9498 loss_val: 0.9925
Epoch: 0150 loss_train: 0.9460 loss_val: 0.9921
Epoch: 0160 loss_train: 0.9479 loss_val: 0.9910
Epoch: 0170 loss_train: 0.9460 loss_val: 0.9930
Epoch: 0180 loss_train: 0.9458 loss_val: 0.9900
Epoch: 0190 loss_train: 0.9428 loss_val: 0.9936
Epoch: 0200 loss_train: 0.9437 loss_val: 0.9919
Epoch: 0210 loss_train: 0.9453 loss_val:

 94%|█████████▍| 16/17 [16:13<00:47, 47.75s/it]

Epoch: 0010 loss_train: 1.0009 loss_val: 0.9967
Epoch: 0020 loss_train: 0.9993 loss_val: 0.9953
Epoch: 0030 loss_train: 0.9901 loss_val: 0.9874
Epoch: 0040 loss_train: 0.9691 loss_val: 0.9659
Epoch: 0050 loss_train: 0.9549 loss_val: 0.9516
Epoch: 0060 loss_train: 0.9452 loss_val: 0.9435
Epoch: 0070 loss_train: 0.9394 loss_val: 0.9386
Epoch: 0080 loss_train: 0.9366 loss_val: 0.9354
Epoch: 0090 loss_train: 0.9347 loss_val: 0.9343
Epoch: 0100 loss_train: 0.9332 loss_val: 0.9334
Epoch: 0110 loss_train: 0.9318 loss_val: 0.9323
Epoch: 0120 loss_train: 0.9309 loss_val: 0.9314
Epoch: 0130 loss_train: 0.9303 loss_val: 0.9300
Epoch: 0140 loss_train: 0.9295 loss_val: 0.9305
Epoch: 0150 loss_train: 0.9292 loss_val: 0.9302
Epoch: 0160 loss_train: 0.9292 loss_val: 0.9288
Epoch: 0170 loss_train: 0.9283 loss_val: 0.9311
Epoch: 0180 loss_train: 0.9283 loss_val: 0.9290
Epoch: 0190 loss_train: 0.9286 loss_val: 0.9279
Epoch: 0200 loss_train: 0.9279 loss_val: 0.9293
Epoch: 0210 loss_train: 0.9282 loss_val:

100%|██████████| 17/17 [17:25<00:00, 61.50s/it]


In [11]:
pd.DataFrame(res)

,0,1,2
0,0.516488,0.664125,0.671594
1,0.704505,0.686349,0.686679
2,0.374600,0.543351,0.546318
3,0.702301,0.726596,0.727581
4,0.563174,0.625798,0.632598
5,0.095591,0.194107,0.219933
6,0.304139,0.427557,0.431296
7,0.036971,0.049754,0.052649
8,0.628163,0.685629,0.693886
9,0.456200,0.574223,0.576181
